# Preparación del Dataset
---

- Se obtienen y preprocesan textos directos del dataset para luego ser persistidos.

In [1]:
import pandas as pd
import os
from document import Document
import util.log as log
from multiprocessing import Process
from multiprocessing import Manager
from multiprocessing import Pool
import pandas as pd
log.init_logger()

In [2]:
corpora = []
errors = []

def process_file(file, corpus):
    log.info('Processing file: {}'.format(file))
    try:
        path = '{}{}'.format(directory, file)
        document = Document(path)
        splitted_file = file.split(' $ ') 
        topic = splitted_file[0]
        title = splitted_file[1]
        corpus.append([path, title, document.string, document.word_count(), document.type_count(), document.sentences,\
                       document.lemmatized_string, document.stemmed_string, document.simple_preprocessed_string, topic, \
                       document.named_entities, document.bigrams, document.trigrams, document.lemmatized_bigrams, \
                       document.lemmatized_trigrams, document.stemmed_bigrams, document.stemmed_trigrams, \
                       document.simple_preprocessed_bigrams, document.simple_preprocessed_trigrams])
    except InvalidDocument:
        errors.append(file)

directory = 'labeled_dataset/'

files = os.listdir(directory)

ps = []

dataframe = None
shared_corpora = None
with Manager() as manager:

    shared_corpora = manager.list()

    for file in files[:2]:
        p = Process(target = process_file, args=(file,shared_corpora))
        p.start()
        ps.append(p)

    for p in ps:
        p.join()

    corpora = list(shared_corpora)


log.info('{} documents were processed. {} documents errored.'.format(len(corpora), len(errors)))

In [3]:
dataframe = pd.DataFrame(data = corpora, columns=['original_path', 'document_title', 'string', 'word_count', 'type_count', 'sentences','lemmatized_text', 'stemmed_text', 'simple_preprocessed', 'topic', 'named_entities', 'tokens_bigrams', 'tokens_trigrams', 'lemmatized_bigrams', 'lemmatized_trigrams', 'stemmed_bigrams', 'stemmed_trigrams', 'simple_preprocessed_bigrams', 'simple_preprocessed_trigrams'])

In [4]:
dataframe

,original_path,document_title,string,word_count,type_count,sentences,lemmatized_text,stemmed_text,simple_preprocessed,topic,named_entities,tokens_bigrams,tokens_trigrams,lemmatized_bigrams,lemmatized_trigrams,stemmed_bigrams,stemmed_trigrams,simple_preprocessed_bigrams,simple_preprocessed_trigrams
0,labeled_dataset/Sistemas emergentes $ TP_6_Wei...,TP_6_Weiss_Gonzalo.pdf,TRABAJO PRÁCTICO N°6 Curso: K5052 Profesor: Al...,480,231,[TRABAJO PRÁCTICO N°6 Curso: K5052 Profesor: A...,"[trabajo, curso, fecha, entregar, nombre, apel...","[trabaj, nombr, email, gmail, com, marketing, ...","[trabajo, curso, fecha, entrega, nombre, apell...",Sistemas emergentes,"[Alejandro Prince Fecha de Entrega, Dictiostel...","[(TRABAJO, PRÁCTICO), (PRÁCTICO, N°6), (N°6, C...","[(TRABAJO, PRÁCTICO, N°6), (PRÁCTICO, N°6, Cur...","[(trabajo, curso), (curso, fecha), (fecha, ent...","[(trabajo, curso, fecha), (curso, fecha, entre...","[(trabaj, nombr), (nombr, email), (email, gmai...","[(trabaj, nombr, email), (nombr, email, gmail)...","[(trabajo, curso), (curso, fecha), (fecha, ent...","[(trabajo, curso, fecha), (curso, fecha, entre..."
1,labeled_dataset/La larga cola $ TP2 - La Larga...,TP2 - La Larga Cola de Chris Anderson corto - ...,Universidad Tecnológica Nacional Facultad Reg...,1234,477,[ Universidad Tecnológica Nacional Facultad Re...,"[sistema, curso, cuat, marketing, nuevo, dr, a...","[cuat, marketing, nuev, dr, ayud, trabaj, col,...","[sistemas, curso, cuat, marketing, nueva, dr, ...",La larga cola,"[Dr. Alejandro Prince Ayudante, Hernán Borré T...","[(Universidad, Tecnológica), (Tecnológica, Nac...","[(Universidad, Tecnológica, Nacional), (Tecnol...","[(sistema, curso), (curso, cuat), (cuat, marke...","[(sistema, curso, cuat), (curso, cuat, marketi...","[(cuat, marketing), (marketing, nuev), (nuev, ...","[(cuat, marketing, nuev), (marketing, nuev, dr...","[(sistemas, curso), (curso, cuat), (cuat, mark...","[(sistemas, curso, cuat), (curso, cuat, market..."
